In [45]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, StratifiedKFold

import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout, MaxPool1D
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, Accuracy
from tensorflow.keras.optimizers import Adam

print(pd.__version__)
print(tf.__version__)
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))

2.0.2
2.12.0
GPUs Available:  []


In [22]:
data_folder = os.path.join('..', '..', 'data', 'mfcc_data', 'grpd_samples')
data_folder

'../../data/mfcc_data/grpd_samples'

In [23]:
phonons = os.listdir(data_folder)

data_path = {}
for phonon in phonons:
    data_path[phonon] = os.path.join(data_folder, phonon)

data_path

{'aa': '../../data/mfcc_data/grpd_samples/aa',
 'yy': '../../data/mfcc_data/grpd_samples/yy',
 'ee': '../../data/mfcc_data/grpd_samples/ee',
 'uu': '../../data/mfcc_data/grpd_samples/uu',
 'oo': '../../data/mfcc_data/grpd_samples/oo'}

In [24]:
# for phonon, phonon_path in data_path.items():
# for type_df in os.listdir(data_path['aa']):
df_paths = os.path.join(data_path['aa'], 'train')

X = []
Y = []
for subject in os.listdir(df_paths):
    df = pd.read_csv(os.path.join(df_paths, subject),  sep=',', index_col=False)

    x = df.loc[:, df.columns != 'asthma_status'].to_numpy()
    y = df.loc[:, df.columns == 'asthma_status'].to_numpy()[0]

    X.append(x)
    Y.append(y)

X = np.stack(X, axis = 0)
Y = np.stack(Y, axis=0)

In [25]:
print(X.shape)
print(Y.shape)

(117, 400, 12)
(117, 1)


In [26]:
df_paths = os.path.join(data_path['aa'], 'val')

Xv = []
Yv = []
for subject in os.listdir(df_paths):
    df = pd.read_csv(os.path.join(df_paths, subject),  sep=',', index_col=False)

    x = df.loc[:, df.columns != 'asthma_status'].to_numpy()
    y = df.loc[:, df.columns == 'asthma_status'].to_numpy()[0]

    Xv.append(x)
    Yv.append(y)

Xv = np.stack(Xv, axis = 0)
Yv = np.stack(Yv, axis=0)

In [27]:
print(Xv.shape)
print(Yv.shape)

(34, 400, 12)
(34, 1)


In [28]:
df_paths = os.path.join(data_path['aa'], 'test')

Xt = []
Yt = []
for subject in os.listdir(df_paths):
    df = pd.read_csv(os.path.join(df_paths, subject),  sep=',', index_col=False)

    x = df.loc[:, df.columns != 'asthma_status'].to_numpy()
    y = df.loc[:, df.columns == 'asthma_status'].to_numpy()[0]

    Xt.append(x)
    Yt.append(y)

Xt = np.stack(Xt, axis = 0)
Yt = np.stack(Yt, axis=0)

In [29]:
print(Xt.shape)
print(Yt.shape)

(17, 400, 12)
(17, 1)


### MODEL BUILDING

In [43]:
def model_maker_aa_o(hp):
    model = keras.Sequential(name='Phonon_to_Asthama_NN')
    model.add(keras.Input(shape=(400, 12)))

    hp_ks = hp.Int(f'fs', min_value=5, max_value=20, step=5)
    hp_pad = hp.Choice('padding', ['valid', 'same'])
    
    # conv1
    model.add(Conv1D(
        hp.Int(f'nf1', min_value=16, max_value=92, step=15),
        kernel_size=hp_ks, padding=hp_pad, activation='relu'
    ))
    model.add(MaxPool1D())

    # conv2
    model.add(Conv1D(
        hp.Int(f'nf2', min_value=16, max_value=92, step=15),
        kernel_size=hp_ks, padding=hp_pad, activation='relu'
    ))
    model.add(MaxPool1D())
    
    # flatten
    model.add(Flatten())

    # dense1
    model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.10))

    # dens2
    model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.10))
    
    # output
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    model.compile(
        loss=BinaryCrossentropy(),
        optimizer=Adam(learning_rate=0.001),
        metrics=[BinaryAccuracy()]
    )

    return  model

In [39]:
def model_maker_aa(hp):
    model = keras.Sequential(name='Phonon_to_Asthama_NN')
    model.add(keras.Input(shape=(419, 12)))
    
    for i in range(2):
        model.add(Conv1D(
            hp.Int(f'nfilters_{i}', min_value=128, max_value=512, step=64),
            kernel_size=(hp.Int(f'fsize_{i}', min_value=5, max_value=20, step=5)),
            padding='valid',
            activation='relu'
        ))
        model.add(MaxPool1D())
    
    model.add(Flatten())
    for i in range(1, hp.Int("num_layers_dense", 2, 3)):
        model.add(Dense(
            hp.Int('layer1', min_value=128, max_value=512, step=64),
            activation='relu'
        ))
        # model.add(Dropout(0.10))
    
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    model.compile(
        loss=BinaryCrossentropy(),
        optimizer=Adam(learning_rate=0.0001),
        metrics=[BinaryAccuracy()]
    )

    return  model

In [41]:
tuner_aa3 = kt.Hyperband(
    model_maker_aa_o,
    objective='val_binary_accuracy',
    max_epochs=40,
    factor=3,
    directory='tuned_models',
    project_name='aa_s3'
)

Model: "Phonon_to_Asthama_NN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 396, 16)           976       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 198, 16)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 194, 16)           1296      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 97, 16)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1552)              0         
                                                                 
 dense_3 (Dense)             (None, 64)       

In [33]:
stop_early = keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy', patience=10
)

In [42]:
tuner_aa3.search(
    X, Y, 
    epochs=40, 
    validation_data=(Xv, Yv), 
    batch_size=1, 
    use_multiprocessing=True, 
    callbacks=[stop_early]
)

Trial 65 Complete [00h 00m 07s]
val_binary_accuracy: 0.6176470518112183

Best val_binary_accuracy So Far: 0.6176470518112183
Total elapsed time: 00h 05m 26s

Search: Running Trial #66

Value             |Best Value So Far |Hyperparameter
10                |20                |fs
valid             |same              |padding
31                |61                |nf1
46                |16                |nf2
160               |128               |layer1
160               |128               |layer2
5                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Model: "Phonon_to_Asthama_NN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 391, 31)           3751      
                                                             

KeyboardInterrupt: 

In [13]:
best_hps = tuner_aa1.get_best_hyperparameters()[0]
best_model = tuner_aa1.hypermodel.build(best_hps)

Model: "Phonon_to_Asthama_NN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 419, 512)          123392    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 209, 512)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 209, 320)          1638720   
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 104, 320)         0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 33280)             0         
                                                                 
 dense (Dense)               (None, 320)      

In [14]:
best_model.save(os.path.join('saved_models', 'supervised', 'aa_s1.h5'))

In [18]:
model = best_model.fit(
    X, Y,
    epochs=100, 
    validation_data=(Xv, Yv), 
    batch_size=8, 
    use_multiprocessing=True, 
    # callbacks=[stop_early]
)

Epoch 1/100
15/15 [==============================] - 3s 169ms/step - loss: 0.0015 - binary_accuracy: 1.0000 - val_loss: 1.1550 - val_binary_accuracy: 0.6765
Epoch 2/100
15/15 [==============================] - 3s 168ms/step - loss: 9.1159e-04 - binary_accuracy: 1.0000 - val_loss: 1.0122 - val_binary_accuracy: 0.5294
Epoch 3/100
15/15 [==============================] - 3s 169ms/step - loss: 7.2297e-04 - binary_accuracy: 1.0000 - val_loss: 1.0854 - val_binary_accuracy: 0.6176
Epoch 4/100
15/15 [==============================] - 3s 168ms/step - loss: 5.1406e-04 - binary_accuracy: 1.0000 - val_loss: 1.0958 - val_binary_accuracy: 0.6176
Epoch 5/100
15/15 [==============================] - 3s 170ms/step - loss: 3.7769e-04 - binary_accuracy: 1.0000 - val_loss: 1.1410 - val_binary_accuracy: 0.6176
Epoch 6/100
15/15 [==============================] - 3s 170ms/step - loss: 2.8416e-04 - binary_accuracy: 1.0000 - val_loss: 1.1459 - val_binary_accuracy: 0.6176
Epoch 7/100
15/15 [===================